# Utilizing multiple devices for training TensorFlow Model

In [36]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [37]:
print(tf.__version__)

2.9.1


In [38]:
import pandas as pd
import numpy as np

In [39]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [40]:
print(gpu_devices)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [41]:
!nvidia-smi

Sun Jul 31 20:58:56 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.73.05    Driver Version: 510.73.05    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  N/A |
| N/A   46C    P8     5W /  N/A |    489MiB /  8192MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  On   | 00000000:0B:00.0 Off |                  N/A |
|  0%   

## Data loading logics
For this example, we will use google scholar data that we crawled in Chapter 2

In [42]:
data = pd.read_csv("~/Production-Ready-Applied-Deep-Learning/Chapter_3/sample_google_scholar.csv")

In [43]:
data = data.dropna()
data.head()

,author_name,email,affiliation,coauthors_names,research_interest
0,Lawrence Holder,wsu.edu,Washington State University,Diane J Cook##William Eberle,artificial_intelligence##machine_learning##dat...
3,Diane J Cook,eecs.wsu.edu,Washington State University,Lawrence Holder##Parisa Rashidi##Sajal K. Das#...,artificial_intelligence##machine_learning##sma...
4,Sumi Helal IEEE Fellow AAAS Fellow IET Fellow ...,cise.ufl.edu,University of Florida,Raja Bose##Darrell Woelk##Diane J Cook##Yousse...,digital_health##smart_homes##internet_of_thing...
5,Hani Hagras,essex.ac.uk,University of Essex,Christian Wagner,explainable_artificial_intelligence##ambient_i...
6,Anupam Joshi,umbc.edu,UMBC,Tim Finin##Yelena Yesha##Lalana Kagal##Dipanja...,data_management##mobile_computing##security##s...


In [44]:
# for features, we will convert first 10 characters of affiliation into a vector of float 
# by dividing each character by maximum axcii number (256)

def convert_first_ten_characters_into_tensor(data):
    first_ten_characters = data[:10]
    converted = [ord(char)/256 for char in first_ten_characters]
    while len(converted) < 10:
        converted.append(0.0)
    return np.array(converted)

converted_affiliation = data['affiliation'].map(convert_first_ten_characters_into_tensor)
affiliation = np.vstack(converted_affiliation.values)
print(affiliation[:5])

[[0.33984375 0.37890625 0.44921875 0.40625    0.41015625 0.4296875
  0.40234375 0.453125   0.43359375 0.4296875 ]
 [0.33984375 0.37890625 0.44921875 0.40625    0.41015625 0.4296875
  0.40234375 0.453125   0.43359375 0.4296875 ]
 [0.33203125 0.4296875  0.41015625 0.4609375  0.39453125 0.4453125
  0.44921875 0.41015625 0.453125   0.47265625]
 [0.33203125 0.4296875  0.41015625 0.4609375  0.39453125 0.4453125
  0.44921875 0.41015625 0.453125   0.47265625]
 [0.33203125 0.30078125 0.2578125  0.26171875 0.         0.
  0.         0.         0.         0.        ]]


In [45]:
# for labels, it will be boolean value; True if email consists of '.edu' and False otherwise
converted_email = data['email'].str.contains('.edu')
labels = converted_email.values
labels[:5]

array([ True,  True,  True, False,  True])

### This concludes this simple preprocessing step. As result, we have features (called affiliation) and labels
In real life, this could be a separate processing job executed for example via spark job. 
Let's save those results as csv file. 

In [46]:
full_df = pd.concat([pd.DataFrame(affiliation),pd.DataFrame(labels)],axis=1, ignore_index=True)
full_df.to_csv('data.csv', index=False, header=True)
full_df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,0.339844,0.378906,0.449219,0.406250,0.410156,0.429688,0.402344,0.453125,0.433594,0.429688,True
1,0.339844,0.378906,0.449219,0.406250,0.410156,0.429688,0.402344,0.453125,0.433594,0.429688,True
2,0.332031,0.429688,0.410156,0.460938,0.394531,0.445312,0.449219,0.410156,0.453125,0.472656,True
3,0.332031,0.429688,0.410156,0.460938,0.394531,0.445312,0.449219,0.410156,0.453125,0.472656,False
4,0.332031,0.300781,0.257812,0.261719,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,True


We can also transform this pandas dataframe to two columns 

In [47]:
full_df['labels'] = full_df.iloc[: , -1].values
full_df['features']= full_df.iloc[: , :-2].values.tolist()
for i in range(11):
    full_df.pop(i)
full_df.to_csv('data_2c.csv', index=False, header=True)

In [48]:
full_df.head()

,labels,features
0,True,"[0.33984375, 0.37890625, 0.44921875, 0.40625, ..."
1,True,"[0.33984375, 0.37890625, 0.44921875, 0.40625, ..."
2,True,"[0.33203125, 0.4296875, 0.41015625, 0.4609375,..."
3,False,"[0.33203125, 0.4296875, 0.41015625, 0.4609375,..."
4,True,"[0.33203125, 0.30078125, 0.2578125, 0.26171875..."


## Utilizing multiple devices for training in TensorFlow

In [49]:
def make_model():
    input_layer = layers.Input(10, name="input_layer")
    x_1 = layers.Dense(128, activation="relu", name="layer1")(input_layer)
    x_2 = layers.Dense(64, activation="relu", name="layer2")(x_1) 
    x_3 = layers.Dense(1, activation="sigmoid", name="layer3")(x_2)
    model2 = keras.Model( input_layer, x_3 )
    return model2

In [50]:
strategy = tf.distribute.MirroredStrategy()
# or 
# mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1", "/gpu:3"])
# if you want to use only specific devices 

with strategy.scope():
    model = make_model()
    model.summary()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 10)]              0         
                                                                 
 layer1 (Dense)              (None, 128)               1408      
                                                                 
 layer2 (Dense)              (None, 64)                8256      
                                                                 
 layer3 (Dense)              (None, 1)                 65        
                                                                 
Total params: 9,729
Trainable params: 9,729
Non-trainable params: 0
_________________________________________________________________


In [51]:
loss = tf.keras.losses.BinaryCrossentropy()

In [52]:
optimizer = tf.keras.optimizers.Adam()

In [53]:
model.compile(loss=loss, optimizer=optimizer)

### Let's make tf.data.Dataset 

In [54]:
train_dataset = tf.data.Dataset.from_tensor_slices((affiliation, labels))
train_dataset = train_dataset.batch(16)

### Let's set auto shard policy to DATA 
see https://www.tensorflow.org/api_docs/python/tf/data/experimental/DistributeOptions for more details

In [55]:
options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.DATA

In [56]:
train_dataset = train_dataset.with_options(options)

In [57]:
model.fit(train_dataset, batch_size=16, epochs=50)

Epoch 1/50
INFO:tensorflow:batch_all_reduce: 6 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 6 all-reduces with algorithm = nccl, num_packs = 1
4/4 [==============================] - 2s 4ms/step - loss: 0.6924
Epoch 2/50
4/4 [==============================] - 0s 3ms/step - loss: 0.6863
Epoch 3/50
4/4 [==============================] - 0s 3ms/step - loss: 0.6838
Epoch 4/50
4/4 [==============================] - 0s 4ms/step - loss: 0.6823
Epoch 5/50
4/4 [==============================] - 0s 4ms/step - loss: 0.6814
Epoch 6/50
4/4 [==============================] - 0s 3ms/step - loss: 0.6803
Epoch 7/50
4/4 [==============================] - 0s 4ms/step - loss: 0.6791
Epoch 8/50
4/4 [==============================] - 0s 5ms/step - loss: 0.6777
Epoch 9/50
4/4 [==============================] - 0s 3ms/step - loss: 0.6763
Epoch 10/50
4/4 [==============================] - 0s 5ms/step - loss: 0.6749
Epoch 11/50
4/4 [==============================] - 0s 4ms/s

In [58]:
strategy = tf.distribute.OneDeviceStrategy(device="/gpu:1")

In [59]:
with strategy.scope():
    model = make_model()
    model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 10)]              0         
                                                                 
 layer1 (Dense)              (None, 128)               1408      
                                                                 
 layer2 (Dense)              (None, 64)                8256      
                                                                 
 layer3 (Dense)              (None, 1)                 65        
                                                                 
Total params: 9,729
Trainable params: 9,729
Non-trainable params: 0
_________________________________________________________________


In [60]:
loss = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [61]:
model.compile(loss=loss, optimizer=optimizer)

In [62]:
model.fit(train_dataset, batch_size=16, epochs=50)

Epoch 1/50
4/4 [==============================] - 0s 3ms/step - loss: 0.6908
Epoch 2/50
4/4 [==============================] - 0s 2ms/step - loss: 0.6828
Epoch 3/50
4/4 [==============================] - 0s 2ms/step - loss: 0.6787
Epoch 4/50
4/4 [==============================] - 0s 2ms/step - loss: 0.6770
Epoch 5/50
4/4 [==============================] - 0s 2ms/step - loss: 0.6758
Epoch 6/50
4/4 [==============================] - 0s 2ms/step - loss: 0.6741
Epoch 7/50
4/4 [==============================] - 0s 2ms/step - loss: 0.6722
Epoch 8/50
4/4 [==============================] - 0s 2ms/step - loss: 0.6702
Epoch 9/50
4/4 [==============================] - 0s 3ms/step - loss: 0.6683
Epoch 10/50
4/4 [==============================] - 0s 3ms/step - loss: 0.6664
Epoch 11/50
4/4 [==============================] - 0s 2ms/step - loss: 0.6644
Epoch 12/50
4/4 [==============================] - 0s 3ms/step - loss: 0.6624
Epoch 13/50
4/4 [==============================] - 0s 2ms/step - loss: 0.

In [63]:
strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")

In [64]:
with strategy.scope():
    model = make_model()
    model.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 10)]              0         
                                                                 
 layer1 (Dense)              (None, 128)               1408      
                                                                 
 layer2 (Dense)              (None, 64)                8256      
                                                                 
 layer3 (Dense)              (None, 1)                 65        
                                                                 
Total params: 9,729
Trainable params: 9,729
Non-trainable params: 0
_________________________________________________________________


In [65]:
loss = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [66]:
model.compile(loss=loss, optimizer=optimizer)

In [67]:
model.fit(train_dataset, batch_size=16, epochs=50)

Epoch 1/50


2022-07-31 20:59:09.813181: W tensorflow/core/framework/dataset.cc:768] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


4/4 [==============================] - 0s 2ms/step - loss: 0.6939
Epoch 2/50
4/4 [==============================] - 0s 2ms/step - loss: 0.6875
Epoch 3/50
4/4 [==============================] - 0s 2ms/step - loss: 0.6842
Epoch 4/50
4/4 [==============================] - 0s 2ms/step - loss: 0.6820
Epoch 5/50
4/4 [==============================] - 0s 2ms/step - loss: 0.6805
Epoch 6/50
4/4 [==============================] - 0s 2ms/step - loss: 0.6794
Epoch 7/50
4/4 [==============================] - 0s 2ms/step - loss: 0.6781
Epoch 8/50
4/4 [==============================] - 0s 2ms/step - loss: 0.6767
Epoch 9/50
4/4 [==============================] - 0s 2ms/step - loss: 0.6752
Epoch 10/50
4/4 [==============================] - 0s 2ms/step - loss: 0.6733
Epoch 11/50
4/4 [==============================] - 0s 2ms/step - loss: 0.6713
Epoch 12/50
4/4 [==============================] - 0s 2ms/step - loss: 0.6693
Epoch 13/50
4/4 [==============================] - 0s 2ms/step - loss: 0.6672
Epoch 